In [7]:
import pandas as pd
import numpy as np
import glob
import os
import mne

# Load data

In [5]:
# example 1_al_ciplv_theta.npy
def parse_filename(filename):
    s = filename.split("_")
    pId = s[0]
    label = s[1]
    method = s[2]
    freq = s[-1].split(".")[0]
    
    return {"pId": pId, "label": label, "method": method, "freq": freq}

def read_file(filename):
    return np.load(filename)

In [53]:
# get all the files in the folder output/
def get_files(folder, filter = None):
    '''
    filter is a dictionary, has 2 keys: method and freq to filter the files
    '''
    files = glob.glob(os.path.join(folder, "*.npy"))
    ret_files = []
    if filter:
        for f in files:
            # get the filename
            filename = os.path.basename(f)
            f_info = parse_filename(filename)
            if (f_info["method"] == filter["method"]) and (f_info["freq"] == filter["freq"]):
                ret_files.append(f)
    else:
        ret_files = files

    return ret_files

In [ ]:
# get the labels for electrodes
fs_dir = mne.datasets.fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)
labels = mne.read_labels_from_annot('fsaverage', parc='aparc',
                                    subjects_dir=subjects_dir)
labels.pop(-1)
label_colors = [label.color for label in labels]

# Preprocessing

In [56]:
filter = {"method": "ciplv", "freq": "theta"}
files = get_files("output", filter)

In [77]:
# take one file and read it
file = files[0]
data = read_file(file)

In [85]:
def feature_extraction(data):
    """
    get lower part of diagonal matrix
    """
    data_lower = data[np.tril_indices(data.shape[0], k=-1)]
    return data_lower

# Data preparation

In [132]:
# get files
methods = ['pli', 'wpli2_debiased', 'ciplv']
freqs = ['delta', 'theta', 'alpha', 'beta', 'gamma']

filter = {"method": "pli", "freq": "theta"}
files = get_files("output", filter)

In [133]:
X = []
Y = []

In [134]:
labels = {"al": 0, "fa":1}

In [135]:
for f in files:
    data = read_file(f)
    X.append(feature_extraction(data))
    Y.append(labels.get(parse_filename(os.path.basename(f))["label"]))

In [136]:
X = np.array(X)
Y = np.array(Y)

In [137]:
# split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Training model

In [140]:
# Use linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# fit model
model = LinearRegression()
model.fit(X, Y)

# evaluate model
Y_pred = model.predict(X)
mse = mean_squared_error(Y, Y_pred)
print("MSE: %.4f" % mse)

# y predicted to binary
Y_pred_binary = np.where(Y_pred > 0, 1, 0)

# print classification report
from sklearn.metrics import classification_report
print(classification_report(Y, Y_pred_binary))


MSE: 0.0000
              precision    recall  f1-score   support

           0       1.00      0.44      0.61        48
           1       0.64      1.00      0.78        48

    accuracy                           0.72        96
   macro avg       0.82      0.72      0.69        96
weighted avg       0.82      0.72      0.69        96



In [139]:
## tensoflow neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD

# create model
model = Sequential()
model.add(Dense(units=256, input_dim=X.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(units=128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])
# fit model with validation data
model.fit(X_train, Y_train, epochs=100, batch_size=8, verbose=1, validation_data=(X_test, Y_test))

# clasification report
Y_pred = model.predict(X_test)
Y_pred_binary = np.where(Y_pred > 0.5, 1, 0)
print(classification_report(Y_test, Y_pred_binary))

Epoch 1/100


c:\Users\work\Anaconda3\envs\kia\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


9/9 [==============================] - 1s 26ms/step - loss: 0.7211 - accuracy: 0.4478 - val_loss: 0.6978 - val_accuracy: 0.3448
Epoch 2/100
9/9 [==============================] - 0s 12ms/step - loss: 0.7111 - accuracy: 0.5224 - val_loss: 0.6970 - val_accuracy: 0.4828
Epoch 3/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6894 - accuracy: 0.6119 - val_loss: 0.6959 - val_accuracy: 0.4828
Epoch 4/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6941 - accuracy: 0.5672 - val_loss: 0.6949 - val_accuracy: 0.5172
Epoch 5/100
9/9 [==============================] - 0s 11ms/step - loss: 0.6987 - accuracy: 0.5224 - val_loss: 0.6954 - val_accuracy: 0.5172
Epoch 6/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6649 - accuracy: 0.5522 - val_loss: 0.6988 - val_accuracy: 0.4828
Epoch 7/100
9/9 [==============================] - 0s 10ms/step - loss: 0.6514 - accuracy: 0.5522 - val_loss: 0.6905 - val_accuracy: 0.4483
Epoch 8/100
9/9 [===============